In [1]:
import os

import imageio
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk

import utoolbox.utils.files as fileutils
from utoolbox.container import Raster
from utoolbox.container.layouts import Volume

Retrieve file list from source directory.

In [77]:
#source_folder = os.path.join(*["data", "20170112_RFiSHp2aLFCYC", "decon", "488"])
source_folder = os.path.join(*["data", "01262018_LightTrigger_U2Os_cell2"])
file_list = fileutils.list_files(
    source_folder,
    name_filters=[
        fileutils.ExtensionFilter('tif'),
        fileutils.SPIMFilter(channel=0)
    ]
)

print("[0] = {}".format(file_list[1]))

[0] = data/01262018_LightTrigger_U2Os_cell2/cell2a_ch1_stack0056_561nm_0336006msec_0005531496msecAbs_decon.tif


Load the image.

In [78]:
image = Raster(file_list[0], layout=Volume, spacing=(.100319, .102, .102))
print("layout={}".format(image.metadata.layout))

layout=<class 'utoolbox.container.layouts.Volume'>


In [ ]:
image = image[::4, ::4, ::4]

image.metadata.layout = Volume
image.metadata.spacing = (.100319*4, .102*4, .102*4)
image.save("data/downsampled.tif")

In [79]:
image_sitk = image.to_sitk()

In [ ]:
mag_filter = sitk.GradientMagnitudeRecursiveGaussianImageFilter()
mag_filter.SetSigma(.01)
mag_filter = mag_filter.Execute(image_sitk)

feature = sitk.BoundedReciprocal(mag_filter)
feature = sitk.Cast(feature, sitk.sitkFloat32)

In [80]:
feature_2 = sitk.BoundedReciprocal(image_sitk)
feature_2 = sitk.Cast(feature_2, sitk.sitkFloat32)

In [89]:
segments = feature_2 < .03

In [91]:
f = sitk.BinaryMorphologicalClosingImageFilter()
f.SetKernelType(sitk.BinaryMorphologicalClosingImageFilter.Ball)
f.SetKernelRadius(5)

segments_f = f.Execute(segments)

In [98]:
f2 = sitk.BinaryFillholeImageFilter()

segments_f2 = f2.Execute(segments_f)

In [92]:
imageio.volwrite("data/tmp.tif", sitk.GetArrayFromImage(segments_f))

In [99]:
f4 = sitk.BinaryMorphologicalOpeningImageFilter()
f4.SetKernelType(sitk.BinaryMorphologicalClosingImageFilter.Ball)
f4.SetKernelRadius(5)

segments_f4 = f.Execute(segments_f2)

In [100]:
imageio.volwrite("data/tmp.tif", sitk.GetArrayFromImage(segments_f4))

In [85]:
f3 = sitk.BinaryContourImageFilter()
segments_f3 = f3.Execute(segments_f2)

In [86]:
imageio.volwrite("data/contour_u2os.tif", sitk.GetArrayFromImage(segments_f3))

Generate distance map with respective to center.

In [28]:
nz, ny, nx = image.shape

seed = sitk.Image(nx, ny, nz, sitk.sitkUInt8)
seed.SetSpacing(image_sitk.GetSpacing())
seed.SetOrigin(image_sitk.GetOrigin())
seed.SetDirection(image_sitk.GetDirection())

seed[nx//2, ny//2, nz//2] = 1

init_sp = sitk.SignedMaurerDistanceMap(seed, insideIsPositive=True, useImageSpacing=True)

In [ ]:
sitk.Show(distance)

In [ ]:
init = sitk.BinaryThreshold(distance, -100, 10)
init = sitk.Cast(init, feature.GetPixelID()) * -1 + 0.5

In [ ]:
sitk.Show(init)

GAC

In [ ]:
gac = sitk.GeodesicActiveContourLevelSetImageFilter()
gac.SetPropagationScaling(.1)
gac.SetCurvatureScaling(.01)
gac.SetAdvectionScaling(1.)
gac.SetMaximumRMSError(.001)
gac.SetNumberOfIterations(1000)

for i in range(10):
    level_set = gac.Execute(init, feature_2)
    imageio.volwrite("data/i_nf_{}.tif".format((i+1)*1000), sitk.GetArrayFromImage(level_set))
    
    print("i={}".format((i+1)*1000))
    print(".. RMS Change : ", gac.GetRMSChange())
    print(".. Elapsed Iterations : ", gac.GetElapsedIterations())
    
    init = level_set

In [ ]:
gac = sitk.GeodesicActiveContourLevelSetImageFilter()
gac.SetPropagationScaling(.1)
gac.SetCurvatureScaling(.01)
gac.SetAdvectionScaling(1.)
gac.SetMaximumRMSError(.001)
gac.SetNumberOfIterations(5000)

level_set = gac.Execute(init, feature)

Upsample

In [ ]:
identity = sitk.Transform()
identity.SetIdentity()

resampler = sitk.ResampleImageFilter()
resampler.SetDefaultPixelValue(0)
resampler.SetTransform(identity)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetOutputSpacing((.100319, .102, .102))
resampler.SetSize((nx, ny, nz))
resampler.SetOutputOrigin((0, 0, 0))
upsample = resampler.Execute(level_set)

In [ ]:
imageio.volwrite("data/upsample.tif", sitk.GetArrayFromImage(upsample))

In [ ]:
sitk.Show(lap_result)

In [ ]:
contour = sitk.BinaryContour(sitk.BinaryThreshold(level_set, -1000, 0 ))
sitk.Show(contour, "Levelset Countour")